# Multithreading (Procesos multiples)

Ahora finalmente estamos listos para comenzar a hablar sobre ejecutar cosas en múltiples procesadores! La mayoría de las computadoras (incluso los teléfonos celulares) en estos días tienen múltiples núcleos o procesadores, por lo que el lugar obvio para comenzar a trabajar con el paralelismo es haciendo uso de aquellos dentro de nuestro proceso de Julia.

Sin embargo, el primer desafío es saber con precisión cuántos "procesadores" tiene. "Procesadores" está entre comillas porque, bueno, es complicado.

In [1]:
versioninfo(verbose = true)

Julia Version 1.8.3
Commit 0434deb161e (2022-11-14 20:14 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
      Ubuntu 22.04.1 LTS
  uname: Linux 5.15.0-53-generic #59-Ubuntu SMP Mon Oct 17 18:53:30 UTC 2022 x86_64 x86_64
  CPU: Intel(R) Core(TM) i7-4790 CPU @ 3.60GHz: 
              speed         user         nice          sys         idle          irq
       #1   800 MHz        299 s         23 s        115 s      10304 s          0 s
       #2  3581 MHz        331 s         18 s        120 s      10552 s          0 s
       #3   800 MHz        353 s         22 s        110 s      10729 s          0 s
       #4  3600 MHz        291 s          7 s        105 s      10718 s          0 s
       #5   800 MHz        259 s         13 s        105 s      10665 s          0 s
       #6   800 MHz        275 s         11 s        114 s      10745 s          0 s
       #7   800 MHz        320 s         14 s        107 s      10202 s          0 s
       #8   800 MHz        294 s         22 s  

In [4]:
]add Hwloc

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
   Installed Hwloc_jll ─ v2.8.0+1
   Installed Hwloc ───── v2.2.0
    Updating `~/.julia/environments/v1.8/Project.toml`
  [0e44f5e4] + Hwloc v2.2.0
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [0e44f5e4] + Hwloc v2.2.0
  [e33a78d0] + Hwloc_jll v2.8.0+1
Precompiling project...
  ✓ Hwloc_jll
  ✓ Hwloc
  2 dependencies successfully precompiled in 18 seconds. 278 already precompiled. 1 skipped during auto due to previous errors.


In [33]:
#; cat /proc/cpuinfo # En máquinas con S.O. Linux

using Hwloc
Hwloc.num_physical_cores()

4

Lo que su computadora informa como la cantidad de procesadores podría no ser lo mismo que la cantidad total de "núcleos". Si bien a veces los procesadores virtuales pueden agregar rendimiento, la paralelización de un cálculo numérico típico sobre estos procesadores virtuales conducirá a un rendimiento significativamente peor porque todavía tienen que compartir gran parte de los aspectos básicos del hardware de cálculo.

¡Julia es algo multiproceso por defecto! Las llamadas BLAS(*Basic Linear Algebra Subroutines*),como la multiplicación de matrices por ejemplo, ya estan separadas en hilos de procesoo:

In [30]:
using BenchmarkTools
A = rand(2000, 2000);
B = rand(2000, 2000);
@btime $A*$B;

  90.839 ms (2 allocations: 30.52 MiB)


¡Esto, de forma predeterminada, ya está usando todos sus núcleos de CPU! Puede ver el efecto cambiando la cantidad de subprocesos (que BLAS admite hacer dinámicamente):

In [34]:
using LinearAlgebra
BLAS.set_num_threads(1)
@btime $A*$B
BLAS.set_num_threads(4)
@btime $A*$B

  318.545 ms (2 allocations: 30.52 MiB)
  89.585 ms (2 allocations: 30.52 MiB)


2000×2000 Matrix{Float64}:
 493.764  511.7    508.259  510.549  …  516.319  517.921  494.235  499.896
 497.313  518.849  516.03   523.746     516.64   525.141  492.888  509.257
 490.165  509.078  505.331  513.398     508.085  519.951  492.625  502.362
 497.596  512.482  501.949  504.023     511.122  521.753  493.886  505.074
 488.939  499.532  498.152  496.361     503.681  510.706  487.794  503.422
 487.473  507.14   492.703  501.106  …  499.099  511.274  478.229  504.224
 497.465  517.869  513.205  515.071     516.291  521.224  495.822  514.05
 516.053  533.57   529.882  528.162     533.811  538.227  511.331  523.337
 469.224  483.383  473.252  479.806     482.659  496.15   465.86   483.869
 497.197  507.107  509.418  506.915     518.051  514.088  491.444  503.385
 495.523  525.249  507.925  520.445  …  513.69   528.267  495.391  508.575
 505.173  515.311  512.38   518.78      513.043  523.536  495.549  513.313
 489.137  506.695  505.148  505.886     510.814  516.288  484.823  498.044

## ¿Cómo luce la implementación de su propio algoritmo de subprocesos?

La compatibilidad con subprocesos múltiples está marcada como "experimental" para Julia 1.0 y está pendiente de una gran renovación para la versión 1.2 o 1.3 de Julia. Los principios básicos serán los mismos, pero debería ser mucho más fácil de usar de manera eficiente.

In [2]:
using .Threads

nthreads()

4


Actualmente, Julia necesita iniciarse sabiendo que tiene habilitada la compatibilidad con subprocesos.
Lo haces con una variable de entorno. Para obtener cuatro subprocesos, inicie Julia con:

JULIA_NUM_THREADS=4 julia

En JuliaBox, esto es un desafío: ¡no tenemos acceso al proceso de lanzamiento!

In [3]:
;env JULIA_NUM_THREADS=4 julia -E 'using .Threads; nthreads()'

4


In [4]:
threadid()

1

In [7]:
A = Array{Union{Int,Missing}}(missing, nthreads())
for i in 1:nthreads()
    A[threadid()] = threadid()
end
A

4-element Vector{Union{Missing, Int64}}:
 1
  missing
  missing
  missing

Pero si le ponemos el prefijo @threads, ¡entonces el cuerpo del ciclo se ejecuta en todos los hilos!

In [8]:
@threads for i in 1:nthreads()
    A[threadid()] = threadid()
end
A

4-element Vector{Union{Missing, Int64}}:
 1
 2
 3
 4

Entonces, intentemos implementar nuestro primer algoritmo sencillo de subprocesos, una suma:

In [9]:
function threaded_sum1(A)
    r = zero(eltype(A))
    @threads for i in eachindex(A)
        @inbounds r += A[i]
    end
    return r
end

A = rand(10_000_000)
threaded_sum1(A)
@time threaded_sum1(A)

  0.308516 seconds (20.00 M allocations: 305.178 MiB, 19.96% gc time)


1.2504660065681003e6

In [10]:
sum(A)
@time sum(A)

  0.005194 seconds (1 allocation: 16 bytes)


5.000803148883128e6

¡Vaya! ¿Qué sucedió? ¡No solo obtuvimos la respuesta incorrecta, sino que fue lento obtenerla!

In [11]:
function threaded_sum2(A)
    r = Atomic{eltype(A)}(zero(eltype(A)))
    @threads for i in eachindex(A)
        @inbounds atomic_add!(r, A[i])
    end
    return r[]
end

threaded_sum2(A)
@time threaded_sum2(A)

  0.821814 seconds (30 allocations: 2.234 KiB)


5.000803148882506e6

¡Bien! Ahora obtuvimos la respuesta correcta (ajustando alguna asociatividad de punto flotante), pero aún es más lento que simplemente hacerlo en 1 núcleo.

In [12]:
threaded_sum2(A) ≈ sum(A)

true

¡Pero sigue siendo lento! ¡Usando *atomics* es mucho más lento que solo agregar números enteros porque constantemente tenemos que ir y verificar qué procesador tiene el último trabajo! Recuerde también que cada subproceso se ejecuta en su propio procesador, ¡y ese procesador también es compatible con SIMD! Bueno, eso si no tuviera que preocuparse por sincronizarse con los otros procesadores...

In [13]:
function threaded_sum3(A)
    r = Atomic{eltype(A)}(zero(eltype(A)))
    len, rem = divrem(length(A), nthreads())
    @threads for t in 1:nthreads()
        rₜ = zero(eltype(A))
        @simd for i in (1:len) .+ (t-1)*len
            @inbounds rₜ += A[i]
        end
        atomic_add!(r, rₜ)
    end
    # catch up any stragglers
    result = r[]
    @simd for i in length(A)-rem+1:length(A)
        @inbounds result += A[i]
    end
    return result
end

threaded_sum3(A)
@time threaded_sum3(A)

  0.003625 seconds (27 allocations: 2.234 KiB)


5.000803148883123e6

Rayos, eso es complicado. También hay un problema:

In [14]:
threaded_sum3(rand(10) .+ rand(10)im) # ¡Prueba con un arreglo de numeros complejos!

LoadError: TypeError: in Atomic, in T, expected T<:Union{Bool, Float16, Float32, Float64, Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}, got Type{ComplexF64}

In [16]:
R = zeros(eltype(A), nthreads())

4-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0

In [17]:
function threaded_sum4(A)
    R = zeros(eltype(A), nthreads())
    @threads for i in eachindex(A)
        @inbounds R[threadid()] += A[i]
    end
    r = zero(eltype(A))
    # sum the partial results from each thread
    for i in eachindex(R)
        @inbounds r += R[i]
    end
    return r
end

threaded_sum4(A)
@time threaded_sum4(A)

  0.003831 seconds (28 allocations: 2.281 KiB)


5.000803148883155e6

Esto sacrifica nuestra capacidad para `@simd`, por lo que es un poco más lento, ¡pero al menos no tenemos que preocuparnos por todos esos índices! Y tampoco necesitamos preocuparnos por lo atómico y podemos admitir nuevamente matrices de cualquier elemento:

In [18]:
threaded_sum4(rand(10) .+ rand(10)im)

5.054601479131771 + 3.595116453284855im


## Conclusiones clave de `threaded_sum`:

* Tenga cuidado con el estado compartido entre hilos: ¡puede dar lugar a respuestas incorrectas!
    * Protéjase usando atomics (or [locks/mutexes](https://docs.julialang.org/en/v1/base/multi-threading/#Synchronization-Primitives-1))
    * Mejor aún: divida el trabajo manualmente de modo que los bucles internos no compartan el estado. `@threads for i in 1:nthreads()` es una expresión útil.
    * Alternativamente, solo use una matriz y solo acceda a los elementos de un solo hilo

# Ser cuidadoso con el *global state*  (¡incluso si no es obvio!)

Otra clase de algoritmo que puede querer paralelizar es un problema de monte-carlo. Dado que cada iteración es un nuevo sorteo aleatorio, y dado que está interesado en observar el resultado agregado, parece que debería prestarse a el paralelismo muy bien!

### Metodo de Monte Carlo
El método de Montecarlo es un método no determinista o estadístico numérico, usado para aproximar expresiones matemáticas complejas y costosas de evaluar con exactitud. El método se llamó así en referencia al Casino de Montecarlo (Mónaco) por ser “la capital del juego de azar”, al ser la ruleta un generador simple de números aleatorios. 

#### Cálculo de $\pi$ por Montecarlo

Consideremos al círculo unitario inscrito en el cuadrado de lado 2 centrado en el origen. Dado que el cociente de sus áreas es $\frac{\pi}{4}$, el valor de $\pi$ puede aproximarse usando Montecarlo de acuerdo al siguiente método:

1. Dibuja un círculo unitario, y al cuadrado de lado 2 que lo inscribe.
2. Lanza un número $n$ de puntos aleatorios uniformes dentro del cuadrado.
3. Cuenta el número de puntos dentro del círculo, es decir, puntos cuya distancia al origen es menor que 1.
4. El cociente de los puntos dentro del círculo dividido entre $n$ es un estimado de $\frac{\pi}{4}$. Multiplica el resultado por 4 para estimar $\pi$.

<img src='Estimacion_de_Pi_por_Montercarlo.gif'>

En este cálculo se tienen que hacer dos consideraciones importantes:

1. Si los puntos no están uniformemente distribuidos, el método es inválido.
2. La aproximación será pobre si solo se lanzan unos pocos puntos. En promedio, la aproximación mejora conforme se aumenta el número de puntos.

In [19]:
using BenchmarkTools

In [20]:
function serialpi(n)
    inside = 0
    for i in 1:n
        x, y = rand(), rand()
        inside += (x^2 + y^2 <= 1)
    end
    return 4 * inside / n
end
serialpi(1)
@time serialpi(100_000_000)

  0.356020 seconds


3.14147064

In [28]:
using .Threads
nthreads()

4

Usemos las técnicas que aprendimos anteriormente para hacer una implementación de subprocesos rápida:

In [22]:
function threadedpi(n)
    inside = zeros(Int, nthreads())
    @threads for i in 1:n
        x, y = rand(), rand()
        @inbounds inside[threadid()] += (x^2 + y^2 <= 1)
    end
    return 4 * sum(inside) / n
end
threadedpi(100_000_000)
@time threadedpi(100_000_000)

  0.093922 seconds (29 allocations: 2.297 KiB)


3.14130468

Bien, ahora ¿por qué no funcionó? ¡Es lento! Veamos la secuencia de números aleatorios que generamos:

In [23]:
import Random
Random.seed!(0)
N = 20000
Rserial = zeros(N)
for i in 1:N
    Rserial[i] = rand()
end
Rserial

20000-element Vector{Float64}:
 0.4056994708920292
 0.06854582438651502
 0.8621408571954849
 0.08597086585842195
 0.6616126907308237
 0.11632735383158599
 0.1093856021447891
 0.7020044441837296
 0.2895098423219379
 0.028549977665983994
 0.538639413965653
 0.8969897902567084
 0.25847781536337067
 ⋮
 0.20453475366744478
 0.5306149811432983
 0.03456372966458843
 0.220988862426311
 0.9249279921301397
 0.007990107701113969
 0.6060173783083965
 0.40485400823870843
 0.9706620597853558
 0.5881340040386561
 0.46443211274507834
 0.8518653025256372

In [24]:
Random.seed!(0)
Rthreaded = zeros(N)
@threads for i in 1:N
    Rthreaded[i] = rand()
end
Rthreaded

20000-element Vector{Float64}:
 0.3337624014920011
 0.08778934169751962
 0.6975330242458738
 0.638933502898981
 0.00864887382901669
 0.5512665131618238
 0.3212281816145137
 0.3463918406165878
 0.6179993182252156
 0.24281669365162994
 0.8852899334338926
 0.6993599111561082
 0.4643813532164971
 ⋮
 0.9373739994909909
 0.00984536404533165
 0.6010075144770451
 0.6985782740022968
 0.7528479401839744
 0.03945273575026298
 0.6860815783160642
 0.6399224816495211
 0.19193799465910422
 0.9785012706620614
 0.9281000132569841
 0.021088421664114843

In [25]:
Set(Rserial) == Set(Rthreaded)

false

In [26]:
indexin(Rserial, Rthreaded)

20000-element Vector{Union{Nothing, Int64}}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 ⋮
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing

¡Ajá, rand() no es seguro para subprocesos! Está mutando (y leyendo) algo global cada vez para averiguar qué sigue. Esto conduce a ralentizaciones y, lo que es peor, sesga la distribución generada de números aleatorios, ¡ya que algunos se repiten!

In [27]:
const ThreadRNG = Vector{Random.MersenneTwister}(undef, nthreads())
@threads for i in 1:nthreads()
    ThreadRNG[Threads.threadid()] = Random.MersenneTwister()
end
function threadedpi2(n)
    inside = zeros(Int, nthreads())
    len, rem = divrem(n, nthreads())
    rem == 0 || error("use a multiple of $(nthreads()), please!")
    @threads for i in 1:nthreads()
        rng = ThreadRNG[threadid()]
        v = 0
        for j in 1:len
            x, y = rand(rng), rand(rng)
            v += (x^2 + y^2 <= 1)
        end
        inside[threadid()] = v
    end
    return 4 * sum(inside) / n
end
threadedpi2(10)
@time threadedpi2(100_000_000)

LoadError: use a multiple of 4, please!